In [13]:
import pandas as pd

pe_data = './datasets/pe_miner_data.pkl'
pe_key = './datasets/pe_miner_key.pkl'


data = pd.read_pickle(pe_data)
key_data = pd.read_pickle(pe_key)


In [14]:
print(len(key_data))

184


In [9]:
for i, key in enumerate(key_data):
    print(key, ':', data[2]['analy'][i])

NameError: name 'key_data' is not defined

In [16]:
import pandas as pd
strings = './datasets/pe_string.pkl'

strings_data = pd.read_pickle(strings)

In [17]:
print(len(strings_data))
print(strings_data[:10])
for str in strings_data[:10]:
    print(str)

29909
[{'CharLowerBuffA', 'GetDriveTypeA', 'DecPos', 'k7__GETORIGI', 'Original', '%__DBCONTIN', '__SETHELPK', 'CreateDirectoryA', 'PreBlock', '---Total Generado por Excel---', 'P.reloc', '1"1*121:1B1J1R1Z1b1j1r1z1', 'Iclinum', 'k7SETFOCUS', 'DelEnd', 'Lock Error', '_CARGO', 'k7MAXCOL', '_PostBlock', 'GetVersion', '4{P:%a', 'invalid', 'k7__KILLREAD', 'Operation too complex', 'k7DBSELECTAR', 'CLIENTE', 'Variable does not exist', 'T*9W&|', 'MEMVAR', 'Create error', '?2?\\?e?u?}?', '$RESULTAD', 'TlsSetValue', 'ExitProcess', 'PostBlock', 'FPUMaskValue', 'CreateMutexA', 'RegQueryValueExA', 'OSCODE', 'GETLIST', '_FILENAME', 'MM/DD/YY', 'nResult', 'MessageBoxA', 'DECPOS', 'GetModuleFileNameA', 'SetCurrentDirectoryA', '1 1$1(1', 'QQQQQQSV', '_CHANGED', 'SEVERITY', 'DBUSEAREA', 'tAN<Dt', '_Changed', 'Range: ', 'ReadFile', 'ERRORMESSA', ' mm... mmm... mmmebs ', 'DESCRIPTIO', 'READEXIT', 'GetThreadLocale', 'RaiseException', 'k7UPDATED', '9_9d9w9', 'GetLogicalDriveStringsA', 'CANRETRY', '<0r"<9v', 

In [7]:
import sys
sys.path.append('../GIPS/core')

from MV2 import MV2
from JIG import JIG
from SG2 import SG2 
from AWL import AWL

In [25]:
import pickle



def main(payload_path, signature_path, stopword_path, virtual_vector_path, big_group_path,
         window_size, K, M, # MV2 파라미터
         thetaJ,  # JIG 파라미터
         vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
        ):
    
    with open(payload_path, 'rb') as f:
        payloads_data = pickle.load(f)

    print(len(payloads_data))
    payloads = [list(x)[0] for x in payloads_data]

    print(len(payloads))

    # 빅 그룹 식별
    minhashed_virtual_vectors = MV2(payloads=payloads, window_size=window_size, K=K, M=M)
    with open(virtual_vector_path, 'wb') as f:
        pickle.dump(minhashed_virtual_vectors, f)

    big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
    with open(big_group_path, 'wb') as f:
        pickle.dump(big_group_indices, f)
    
    big_group_payloads = []
    non_big_group_paylaods = []

    for idx, payload in enumerate(payloads):
        if idx in big_group_indices:
            big_group_payloads.append(payload)
        else:
            non_big_group_paylaods.append(payload)
    
    # 시그니처 생성
    cluster_signatures = SG2(payloads=big_group_payloads, window_size=window_size, vector_size=vector_size, 
                             eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    
    with open(signature_path, 'wb') as f:
        pickle.dump(cluster_signatures, f)
    
    stopwords = AWL(payloads=non_big_group_paylaods, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

    with open(stopword_path, 'wb') as f:
        pickle.dump(stopwords, f)


if __name__ == '__main__':

    file_path = '../PE/datasets/pe_string.pkl'
    virtual_vector_path = f'./res/PE_virtual_vector.pkl'
    big_group_indices_path = f'./res/PE_big_group.pkl'
    signature_path = f'./res/PE_signature.pkl'
    stopword_path = f'./res/PE_stopword.pkl'

    K = 64
    M = 2 ** 14
    thetaJ = 0.6
    window_size = 4
    vector_size = 512
    eps = 0.4
    minpts = 5
    ngram = 4
    hh1_size = 3000
    hh2_size = 3000
    hh3_size = 3000
    ratio = 0.1

    main(payload_path=file_path, signature_path=signature_path, stopword_path=stopword_path,
         virtual_vector_path=virtual_vector_path, big_group_path=big_group_indices_path,
         window_size=window_size, vector_size=vector_size, K=K, M=M, thetaJ=thetaJ,
         eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh2_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    

29909
29909
make minhashed vector


100%|██████████| 29909/29909 [00:08<00:00, 3580.49it/s]


checking big group


100%|██████████| 29909/29909 [02:25<00:00, 205.38it/s]


chunking


...

start DBSCAN


end DBSCAN
make signature


100%|██████████| 61/61 [00:00<00:00, 381.95it/s]

end signature
start whtie list


end white list


In [26]:
import pickle
with open('./res/PE_signature.pkl', 'rb') as f:
    signature = pickle.load(f)
    
with open('./res/PE_stopword.pkl', 'rb') as f:
    stopword = pickle.load(f)
    
with open('./res/PE_big_group.pkl', 'rb') as f:
    big_group = pickle.load(f)

In [27]:
print(len(big_group))

22566


[0, 1, 2, 3, 4, 5, 6, 7, 12, 18, 23, 26, 27, 31, 35, 46, 48, 49, 50, 61, 66, 69, 76, 82, 84, 86, 92, 94, 95, 99, 101, 103, 107, 110, 112, 113, 114, 115, 117, 118, 120, 122, 126, 130, 132, 141, 142, 146, 148, 151, 158, 159, 161, 166, 168, 169, 170, 173, 177, 183, 186, 189, 192, 196, 198, 199, 200, 201, 205, 206, 207, 209, 216, 220, 223, 224, 227, 228, 229, 233, 234, 237, 238, 239, 241, 244, 249, 254, 256, 258, 260, 268, 270, 273, 274, 276, 277, 284, 288, 291]


In [11]:
print('SIGNAURE')

for i in signature.values():
    for j in i:
        print(j, end=" ")
    print()
    
print('STOPWORD')
for i in stopword:
    print(i)

SIGNAURE
('$&-[-o', 832) 
('-H@{c!', 340) 
('d!`\\rr', 1008) 
('TRegistryU', 966) ('__vbaVarLateMemCallLd', 409) ('L8!H@w', 99) 
('open file', 607) 
('RtlDecompressBuffer', 510) 
('l(lBt]', 312) ('Inverflow', 156) ('symname(', 81) ('{ezxyxf+:$x', 12) 
('l][tSc', 2184) 
('module_main', 919) 
('FPlt68', 924) 
('v: 1.31', 742) ('CreateProcessA', 31) 
('StartMailWorm', 852) 
('TComponent', 1040) ('__vbaFreeStrList', 351) 
('           ', 605) 
('Wm>sdw', 701) ('Websit', 478) 
('       ', 635) 
('GetLocaleInfoEx', 465) ('.adata', 288) ('0v) `@', 45) 
('"?O=N(', 817) 
('     ', 330) 
('1B2R2_2h2n2x2~2', 344) 
('HbB<!T', 350) 
('_t$@SDI', 241) 
('Iac}JDTb2`A@', 244) 
('GetLongPathNameA', 227) ('FindNextFileA', 150) 
('EncodePointer', 432) 
('%U|mBk', 260) 
('4N:.a:', 169) 
('Init*K4', 269) 
('1 1$1(1,1014181<1@1D1H1L1P1T1X1\\1`1d1h1l1p1t1x1|1', 236) 
('BBBBRV', 110) 
("d,I'0ld6", 184) 
('@A_A^_^]', 156) 
('`-Q_Dv', 194) 
('\\ShowYJ', 166) 
('_cxx1', 89) 
('iiiiiiiiiii', 176) ('a2AQVCO', 14) 


In [12]:
filters = [i[0] for i in stopword if i[1] > 99]
for cluster, sig in signature.items():
    for token in sig:
        if token[1] not in filters:
            print(token)

('$&-[-o', 832)
('-H@{c!', 340)
('d!`\\rr', 1008)
('TRegistryU', 966)
('__vbaVarLateMemCallLd', 409)
('L8!H@w', 99)
('open file', 607)
('RtlDecompressBuffer', 510)
('l(lBt]', 312)
('Inverflow', 156)
('symname(', 81)
('{ezxyxf+:$x', 12)
('l][tSc', 2184)
('module_main', 919)
('FPlt68', 924)
('v: 1.31', 742)
('CreateProcessA', 31)
('StartMailWorm', 852)
('TComponent', 1040)
('__vbaFreeStrList', 351)
('           ', 605)
('Wm>sdw', 701)
('Websit', 478)
('       ', 635)
('GetLocaleInfoEx', 465)
('.adata', 288)
('0v) `@', 45)
('"?O=N(', 817)
('     ', 330)
('1B2R2_2h2n2x2~2', 344)
('HbB<!T', 350)
('_t$@SDI', 241)
('Iac}JDTb2`A@', 244)
('GetLongPathNameA', 227)
('FindNextFileA', 150)
('EncodePointer', 432)
('%U|mBk', 260)
('4N:.a:', 169)
('Init*K4', 269)
('1 1$1(1,1014181<1@1D1H1L1P1T1X1\\1`1d1h1l1p1t1x1|1', 236)
('BBBBRV', 110)
("d,I'0ld6", 184)
('@A_A^_^]', 156)
('`-Q_Dv', 194)
('\\ShowYJ', 166)
('_cxx1', 89)
('iiiiiiiiiii', 176)
('a2AQVCO', 14)
('Content-Type: text/plain', 66)
('*BTj1oR', 